In [320]:
import tensorflow as tf
from tensorflow import keras
import dlib
import cv2
import mtcnn
from mtcnn.mtcnn import MTCNN


In [321]:
model = MTCNN()

In [322]:
imag = cv2.imread('./myImage2.jpeg')

In [323]:
cv2.imshow('image',imag)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [324]:
from PIL import Image as im
import numpy as np
import matplotlib.pyplot as plt


In [325]:
def normalise(original_image,face_with_landmarks, desired_left_eye=(0.33,0.33), desired_nose=(0.5,0.5)):
    
    faceX, faceY, face_width, face_height = face_with_landmarks['box']
    left_eye = np.array(face_with_landmarks['keypoints']['left_eye'])
    right_eye = np.array(face_with_landmarks['keypoints']['right_eye'])
    eye_center = (left_eye + right_eye)/2
    
    nose = np.array(face_with_landmarks['keypoints']['nose'])
    nose_distance = np.sum((eye_center - nose)**2)
    # print("nose_distance ",nose_distance)
    eyes_distance = np.sum((left_eye - right_eye)**2)
    # print("eyes_distance ",eyes_distance)
    input_points = np.float32([left_eye, right_eye, nose])

    scaling_factor = nose_distance/eyes_distance 
    # print("scaling factor ",scaling_factor)
    desired_nose = np.array([0.5,2*desired_left_eye[1]-(1-2*desired_left_eye[0])*scaling_factor])
    # print("desired nose ",desired_nose)

    desired_right_eye = np.array([1-desired_left_eye[0],desired_left_eye[1]])*np.array([face_width,face_height])
    desired_left_eye = np.array(desired_left_eye)*np.array([face_width,face_height])

    desired_nose = desired_nose*np.array([face_width,face_height])

    output_points = np.float32([desired_left_eye, desired_right_eye, desired_nose]) 

    M = cv2.getAffineTransform(input_points, output_points)
    
    dst = cv2.warpAffine(original_image, M,(face_width,face_height))
    return dst 




In [326]:
faces = model.detect_faces(imag)
print(faces)
cv2.imshow("cropped image",imag)
cv2.waitKey(0)
cv2.destroyAllWindows()


[{'box': [277, 457, 264, 336], 'confidence': 0.9893486499786377, 'keypoints': {'left_eye': (345, 567), 'right_eye': (468, 585), 'nose': (384, 638), 'mouth_left': (332, 703), 'mouth_right': (431, 722)}}]


In [327]:
for face in faces:
    newImage =normalise(imag,face)
    print(newImage.shape)
    cv2.imshow("cropped image",newImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

nose_distance  4350.25
eyes_distance  15453
scaling factor  0.28151491619750213
desired nose  [0.5        0.56428493]
(336, 264, 3)


In [328]:
from keras_facenet import FaceNet

In [346]:
known_embeddings = []
names = []

embedder = FaceNet()
imag = cv2.imread('./myImage.jpeg')
faces = model.detect_faces(imag)
for face in faces:
    croppedFace = normalise(imag,face)
    detections = embedder.extract(imag)
    known_embeddings.append(detections[0]['embedding'])
    names.append("pranay")

imag = cv2.imread('./joji.jpg')
faces = model.detect_faces(imag)
for face in faces:
    croppedFace = normalise(imag,face)
    detections = embedder.extract(imag)
    known_embeddings.append(detections[0]['embedding'])
    names.append("joji")

known_embeddings = np.array(known_embeddings)
print(known_embeddings.shape)
names = np.array(names)


nose_distance  9518.5
eyes_distance  32050
scaling factor  0.29698907956318255
desired nose  [0.5        0.55902371]
nose_distance  564.5
eyes_distance  2450
scaling factor  0.2304081632653061
desired nose  [0.5        0.58166122]
(2, 512)


In [348]:
vid=cv2.VideoCapture(0)
processFrame = 0
while(True):
    ret,frame = vid.read()
    # print(frame.shape)
    if processFrame%2 == 0:
        faces = model.detect_faces(frame)
        for face in faces:
            croppedFace = normalise(frame, face)
            croppedFace = cv2.resize(croppedFace, (160,160))
            embeddings_from_frame = embedder.extract(croppedFace)
            # print(embeddings_from_frame)
            if embeddings_from_frame != []:
                embeddings_from_frame = np.array(embeddings_from_frame[0]['embedding'])
                print(embeddings_from_frame.shape)
                index = np.argmin(np.sum((known_embeddings-embeddings_from_frame)**2,axis=1))
                x,y,w,h = face['box']
                frame = cv2.rectangle(frame,(x,y),(x+width,y+height),(255,0,0),2)            
                frame = cv2.putText(frame,names[index],(x+2,y+height),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2, cv2.LINE_AA)
            cv2.imshow("frame",frame)
    processFrame = processFrame + 1
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

nose_distance  1017.25
eyes_distance  5185
scaling factor  0.19619093539054966
desired nose  [0.5        0.59329508]
nose_distance  966.25
eyes_distance  5501
scaling factor  0.1756498818396655
desired nose  [0.5        0.60027904]
nose_distance  874.25
eyes_distance  4105
scaling factor  0.21297198538367845
desired nose  [0.5        0.58758952]
(512,)
nose_distance  843.25
eyes_distance  4225
scaling factor  0.19958579881656804
desired nose  [0.5        0.59214083]
(512,)
nose_distance  841.25
eyes_distance  4229
scaling factor  0.19892409553085835
desired nose  [0.5        0.59236581]
(512,)
nose_distance  1093.0
eyes_distance  4356
scaling factor  0.25091827364554636
desired nose  [0.5        0.57468779]
(512,)
nose_distance  870.5
eyes_distance  4226
scaling factor  0.2059867486985329
desired nose  [0.5        0.58996451]
(512,)
nose_distance  996.25
eyes_distance  4357
scaling factor  0.2286550378700941
desired nose  [0.5        0.58225729]
(512,)
nose_distance  934.25
eyes_distan

In [ ]:

.